이건 진짜 마지막!

In [1]:
#에러 코드 핸들링
import os
import warnings

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

#디렉토리 -> numpy 변환
import cv2

# 이미지 -> 디렉토리 및 라벨링
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical

#모델링
from keras.models import Sequential,load_model
from tensorflow.keras.layers import  Input, Conv2D, MaxPooling2D, Dense, Flatten, experimental, Dropout

import pandas as pd 




os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
warnings.filterwarnings(action='ignore')



# 데이터 증폭
train_data_augmentation = ImageDataGenerator(
    validation_split = 0.2,
    rotation_range=10,
    zoom_range = 0.3,
    width_shift_range=0.2,
    height_shift_range=0.2)

#train 데이터 설정 
train_data = train_data_augmentation.flow_from_directory(
        './images2/train',
        target_size = (300,300),
        batch_size=150,
        #color_mode ='rgb',
        classes = ['food','interior', 'exterior'],
        shuffle =True,
        class_mode='categorical')


# validation 데이터 설정
validation_data = train_data_augmentation.flow_from_directory(
        './images2/train',
        target_size = (300,300),
        batch_size=150,
        classes = ['food','interior', 'exterior'],
        shuffle =True,
        class_mode='categorical',
        subset='validation')

#test 데이터 설정
test_data = ImageDataGenerator().flow_from_directory(
        './images2/test',
        classes = ['food','interior', 'exterior'],
        target_size=(300,300),
        batch_size=100,
        shuffle =True,
        class_mode='categorical')

Found 36000 images belonging to 3 classes.
Found 7200 images belonging to 3 classes.
Found 9000 images belonging to 3 classes.


In [2]:
def plot_loss_curve(history):
    plt.figure(figsize = (5,3))
    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    plt.title('model loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['train','test'], loc = 'upper right')
    plt.show


model = load_model('Model-19')

# 5. 모델 평가하기

print("-- Evaluate --")
scores = model.evaluate_generator(test_data, steps=5)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))


# 6. 모델 사용하기

print("-- Predict --")
output = model.predict_generator(test_data, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_data.class_indices)
print(output)

plot_loss_curve(history.history)

print(history.history)
print('train loss =', history.history['loss'][-1])
print('validation loss =', history.history['val_loss'][-1])

In [3]:
X,y = test_data.next()

In [4]:
def print_class(Num):
    class_of_Image = '음식' if Num == 0 else '실내' if Num==1 else '실외'
    return class_of_Image


In [5]:

def predict_image_sample_Noprint(model,X_test, y_test,test_id= -1 ):

    if test_id <0 :

        from random import randrange
        test_sample_id = randrange(len(y))

    else:
        test_sample_id = test_id

    test_image =X_test[test_sample_id]


    #이미지로 만들기
#    pixels = np.array(test_image,np.int32)
#    plt.imshow(pixels)
#     plt.show()
    
    test_image = test_image.reshape(1,300,300,3)
    
#     print(test_data.class_indices)
    y_actual = y_test[test_sample_id]
#     print( y_actual)
    y_actual_num = np.argmax(y_actual)
    actual_class = print_class(y_actual_num)
#     print('실제 클래스 = ', actual_class)

    
    y_pred = model.predict(test_image)
    #print("y_pred = ", y_pred)
    y_pred_num = np.argmax(y_pred, axis = 1)[0]
    predict_class = print_class(y_pred_num)
#     print('예측 클래스 = ', predict_class)
    if y_pred_num != y_actual_num:
        #print("sample %d is wrong!" %test_sample_id)
        with open("wrong_samples.txt", "a") as errfile:
            pass
    else:
        pass
        #print("sample %d is correct!" %test_sample_id)

    
    return y_actual_num, y_pred_num , y_pred

In [6]:

def predict_image_sample_print(model,X_test, y_test, Num = 1,test_id= -1 ):

    if test_id <0 :

        from random import randrange
        test_sample_id = randrange(len(y))

    else:
        test_sample_id = test_id

    test_image =X_test[test_sample_id]


    #이미지로 만들기
    pixels = np.array(test_image,np.int32)
    plt.imshow(pixels)
    plt.show()
    
    test_image = test_image.reshape(1,300,300,3)
    
    print(test_data.class_indices)
    y_actual = y_test[test_sample_id]
    print( y_actual)
    y_actual_num = np.argmax(y_actual)
    actual_class = print_class(y_actual_num)
    print('실제 클래스 = ', actual_class)

    
    y_pred = model.predict(test_image)
    print("y_pred = ", y_pred)
    y_pred_num = np.argmax(y_pred, axis = 1)[0]
    predict_class = print_class(y_pred_num)
    print('예측 클래스 = ', predict_class)
    
    
    if y_pred_num != y_actual_num:
        print("sample %d is wrong!" %test_sample_id)
        with open("wrong_samples.txt", "a") as errfile:
            print("%d"%test_sample_id, file=errfile)
    else:
        print("sample %d is correct!" %test_sample_id)
    
    
    
    
    return y_actual_num, y_pred_num

In [9]:
if __name__ == '__main__' :
    for j in range(4,15):
        model = load_model('model-'+str(j))
        X_test , y_test = X,y
        corr,count = 0,0

        for i in range(len(y)):
            count +=1
            y_act, y_pre, y_pred= predict_image_sample_Noprint(model,X_test, y_test)
            if y_act == y_pre :
                corr +=1 
            else:continue
                
        print('Model-{},총횟수 = {} , 맞춘횟수={} , 정확도={}      '.format(j,count,corr, (corr/count)*100 ) ,y_pred )


Model-4,총횟수 = 100 , 맞춘횟수=13 , 정확도=13.0       [[1. 0. 0.]]
Model-5,총횟수 = 100 , 맞춘횟수=40 , 정확도=40.0       [[0. 1. 0.]]
Model-6,총횟수 = 100 , 맞춘횟수=29 , 정확도=28.999999999999996       [[6.5977375e-14 9.9889803e-01 1.1019910e-03]]
Model-7,총횟수 = 100 , 맞춘횟수=34 , 정확도=34.0       [[0.0000000e+00 9.9970144e-01 2.9855303e-04]]
Model-8,총횟수 = 100 , 맞춘횟수=60 , 정확도=60.0       [[0. 0. 1.]]
Model-9,총횟수 = 100 , 맞춘횟수=71 , 정확도=71.0       [[0. 1. 0.]]
Model-10,총횟수 = 100 , 맞춘횟수=28 , 정확도=28.000000000000004       [[0. 0. 1.]]
Model-11,총횟수 = 100 , 맞춘횟수=35 , 정확도=35.0       [[1. 0. 0.]]
Model-12,총횟수 = 100 , 맞춘횟수=27 , 정확도=27.0       [[0. 0. 1.]]
Model-13,총횟수 = 100 , 맞춘횟수=31 , 정확도=31.0       [[0. 0. 1.]]
Model-14,총횟수 = 100 , 맞춘횟수=57 , 정확도=56.99999999999999       [[0. 0. 1.]]


In [12]:
if __name__ == '__main__' :
    for j in range(2,6):
        model = load_model('model-16-'+str(j))
        X_test , y_test = X,y
        corr,count = 0,0

        for i in range(len(y)):
            count +=1
            y_act, y_pre, y_pred= predict_image_sample_Noprint(model,X_test, y_test)
            if y_act == y_pre :
                corr +=1 
            else:continue
                
        print('Model-{},총횟수 = {} , 맞춘횟수={} , 정확도={}      '.format(j,count,corr, (corr/count)*100 ) ,y_pred )


Model-2,총횟수 = 100 , 맞춘횟수=27 , 정확도=27.0       [[0.000000e+00 2.960595e-26 1.000000e+00]]
Model-3,총횟수 = 100 , 맞춘횟수=34 , 정확도=34.0       [[0. 0. 1.]]
Model-4,총횟수 = 100 , 맞춘횟수=38 , 정확도=38.0       [[0. 0. 1.]]
Model-5,총횟수 = 100 , 맞춘횟수=45 , 정확도=45.0       [[1.6103559e-03 2.6529972e-13 9.9838972e-01]]


In [10]:
if __name__ == '__main__' :
    for j in range(16,22):
        model = load_model('model-'+str(j))
        X_test , y_test = X,y
        corr,count = 0,0

        for i in range(len(y)):
            count +=1
            y_act, y_pre, y_pred= predict_image_sample_Noprint(model,X_test, y_test)
            if y_act == y_pre :
                corr +=1 
            else:continue
                
        print('Model-{},총횟수 = {} , 맞춘횟수={} , 정확도={}      '.format(j,count,corr, (corr/count)*100 ) ,y_pred )


Model-16,총횟수 = 100 , 맞춘횟수=50 , 정확도=50.0       [[0. 0. 1.]]
Model-17,총횟수 = 100 , 맞춘횟수=24 , 정확도=24.0       [[0.0000000e+00 4.1638892e-36 1.0000000e+00]]
Model-18,총횟수 = 100 , 맞춘횟수=48 , 정확도=48.0       [[1.0000000e+00 2.9114954e-33 0.0000000e+00]]
Model-19,총횟수 = 100 , 맞춘횟수=69 , 정확도=69.0       [[0.9975635  0.00243649 0.        ]]
Model-20,총횟수 = 100 , 맞춘횟수=36 , 정확도=36.0       [[0.34350306 0.33154276 0.32495415]]
Model-21,총횟수 = 100 , 맞춘횟수=42 , 정확도=42.0       [[0.3455544  0.3322728  0.32217276]]


In [13]:
if __name__ == '__main__' :
    Listtt = ['18-3','2018-2','2020','2020-1215-11시','2020-1215-12시','201814132','X','X2']
    for j in Listtt:
        model = load_model('model-'+j)
        X_test , y_test = X,y
        corr,count = 0,0

        for i in range(len(y)):
            count +=1
            y_act, y_pre, y_pred= predict_image_sample_Noprint(model,X_test, y_test)
            if y_act == y_pre :
                corr +=1 
            else:continue
                
        print('Model-{},총횟수 = {} , 맞춘횟수={} , 정확도={}      '.format(j,count,corr, (corr/count)*100 ) ,y_pred )


Model-18-3,총횟수 = 100 , 맞춘횟수=42 , 정확도=42.0       [[3.3531350e-16 9.5407218e-01 4.5927856e-02]]
Model-2018-2,총횟수 = 100 , 맞춘횟수=43 , 정확도=43.0       [[1. 0. 0.]]
Model-2020,총횟수 = 100 , 맞춘횟수=19 , 정확도=19.0       [[0.33106902 0.33419204 0.33473903]]
Model-2020-1215-11시,총횟수 = 100 , 맞춘횟수=26 , 정확도=26.0       [[0.3150088  0.34283704 0.34215415]]
Model-2020-1215-12시,총횟수 = 100 , 맞춘횟수=27 , 정확도=27.0       [[6.2605098e-17 7.8941930e-06 9.9999213e-01]]
Model-201814132,총횟수 = 100 , 맞춘횟수=40 , 정확도=40.0       [[0.86444366 0.0015804  0.13397591]]
Model-X,총횟수 = 100 , 맞춘횟수=58 , 정확도=57.99999999999999       [[0.3190809  0.34213278 0.3387863 ]]
Model-X2,총횟수 = 100 , 맞춘횟수=53 , 정확도=53.0       [[1. 0. 0.]]
